<h1>The Battle of the Neighborhoods - Week 1</h1>

<h2>Data </h2>

This project is based on Foursqare API. The coordinates of each city were specified using *geopy library*.
Venues matching defined categories - Jazz and rock clubs, music stores and their details were found using Foursqaure look ups.

<h2>Data examples</h2>

<h3>geopy</h3>

In [99]:
from geopy.geocoders import Nominatim #import the function
geolocator = Nominatim(user_agent="capstone") #define geolocator
location = geolocator.geocode('New York City') #look up the city
location

Location(New York, United States of America, (40.7127281, -74.0060152, 0.0))

In [100]:
latitude = location.latitude
longitude = location.longitude
"{}, {}".format((latitude),(longitude))

'40.7127281, -74.0060152'

Thanks to geopy we can easly find needed coordiantes.

<h3>Foursqare</h3>

In [101]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
###
CLIENT_ID = 'JQYNPSOQXI12NK311020I2K4FAKMKEDV25CMLKTGSEUEVMUR' # your Foursquare ID
CLIENT_SECRET = 'XRECAKXLEOK324BN2Z3KDYNK1NBJ3UDYYS45PQAJ2LIM0BFX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
###
LIMIT = 100 
cities = ['Cracow']
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4bf58dd8d48988d1e7931735,4bf58dd8d48988d1e9931735") 
    results[city] = requests.get(url).json()
df_venues={}
for city in cities:
    venues = json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address','venue.id', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address','id', 'Lat', 'Lng']

Your credentails:
CLIENT_ID: JQYNPSOQXI12NK311020I2K4FAKMKEDV25CMLKTGSEUEVMUR
CLIENT_SECRET:XRECAKXLEOK324BN2Z3KDYNK1NBJ3UDYYS45PQAJ2LIM0BFX


In [102]:
df_venues['Cracow'].head(2)

,Name,Address,id,Lat,Lng
0,Harris Piano Jazz,Rynek Główny 28,4bbf90a2b492d13a4fdfa260,50.061703,19.935660
1,Jazz Rock Cafe,Sławkowska 12,4befe97724f19c747926f983,50.063866,19.937492


First Foursqare request returns music clubs in defined cities. With ids of the venues we can look up more details about them:

In [103]:
lookup_id = df['id'][0]
URL='https://api.foursquare.com/v2/venues/{}/tips'.format(lookup_id)
venue = requests.get(URL).json()
lookup_id

'4bbf90a2b492d13a4fdfa260'

In [104]:
URL = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}'.format(
        lookup_id,
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        ) 
grab = requests.get(URL).json()

In [105]:
grab['response']['venue']['rating']

7.9

Location and rating data will be used to determine the best city for young artists.